# Nanopore data analysis
## Useful commands and workflow by Piroon Jenjaroenpun

### 0. set up python environment for data analysis
####  Anaconda installation
Most of software could be installed via Conda. Conda is a cross-platform, language-agnostic binary package manager. It is the package manager used by Anaconda installations. 

**install program from Anaconda (Please use terminal)**
```bash
bash programs/Anaconda3-2019.07-MacOSX-x86_64.sh
conda config --add channels defaults
conda config --add channels conda-forge
conda config --add channels bioconda
conda config --add channels r
conda install nb_conda jupyter_contrib_nbextensions jupyter \
  rpy2 ipykernel r-irkernel r-devtools parallel
```


#### Install conda environment

**Please use Terminal**
```bash
conda create -n ont_assembly python=3 biopython nanofilt nanoplot porechop samtools minimap2 miniasm racon assembly-stats r-ggplot2 r-plotly r-optparse bioawk 
conda create -n unicycler unicycler
conda create -n flye fly
conda create -n bandage bandage
ln -s ~/.conda/envs/bandage/bin/Bandage /home/adnano/.conda/envs/ont_assembly/bin/
ln -s ~/.conda/envs/unicycler/bin/unicycler /home/adnano/.conda/envs/ont_assembly/bin/
ln -s ~/.conda/envs/flye/bin/flye /home/adnano/.conda/envs/ont_assembly/bin/

```

**Activate environment**
```bash
source activate ont_assembly
```
**Install igv-jupyter**
```bash
pip install igv-jupyter
# To install to configuration in your home directory
jupyter serverextension enable --py igv
jupyter nbextension install --py igv
jupyter nbextension enable --py igv
# If using a virtual environment
jupyter serverextension enable --py igv --sys-prefix
jupyter nbextension install --py igv --sys-prefix
jupyter nbextension enable --py igv --sys-prefix
```

---

### 1.basecalled
Guppy is ONT's official command-line basecaller. Guppy basecaller is used to convert raw signal to DNA/RNA sequence. Guppy can be downloaded from the Nanopore community, but you'll need an account to log in.

#### run Guppy Basecalling

<pre>
Guppy Basecalling Software, (C) Oxford Nanopore Technologies, Limited. 
Version 2.3.5+53a111f

Usage (basic command):

With config file:
  guppy_basecaller -i [input path] -s [save path] -c [config file] [options]
With flowcell and kit name:
  guppy_basecaller -i [input path] -s [save path] --flowcell [flowcell name]
    --kit [kit name]
List supported flowcells and kits:
  guppy_basecaller --print_workflows
Use server for basecalling:
  guppy_basecaller -i [input path] -s [save path] -c [config file]
    --port [server address] [options] 

Command line parameters:
  --print_workflows                 Output available workflows.
  --flowcell arg                    Flowcell to find a configuration for
  --kit arg                         Kit to find a configuration for
  --qscore_filtering                Enable filtering of reads into PASS/FAIL
                                    folders based on min qscore.
  --min_qscore arg                  Minimum acceptable qscore for a read to be
                                    filtered into the PASS folder
  -r [ --recursive ]                Search for input files recursively.
  -q [ --records_per_fastq ] arg    Maximum number of records per fastq file, 0
                                    means use a single file (per worker, per
                                    run id).
  -h [ --help ]                     produce help message
  -v [ --version ]                  print version number
  -c [ --config ] arg               Config file to use
  -d [ --data_path ] arg            Path to use for loading any data files the 
                                    application requires.
</pre>

In [1]:
%%bash
in_fast5Dir=data/0.fast5
out_basecalledDir=result/1.basecalled
time guppy_basecaller -i $in_fast5Dir -s $out_basecalledDir \
       --recursive --records_per_fastq 0 \
       --flowcell FLO-MIN106 --kit SQK-RAD004 \
       --qscore_filtering --min_qscore 8 \
       --num_callers 1 --cpu_threads_per_caller 7 \
       --compress_fastq

echo "Basecalling is done."
echo "------------------"

ONT Guppy basecalling software version 3.1.5+781ed575
config file:        /Users/piroonjenjaroenpun/Desktop/project/snakefiles/guppy/program/ont-guppy-cpu_3.1.5/data/dna_r9.4.1_450bps_hac.cfg
model file:         /Users/piroonjenjaroenpun/Desktop/project/snakefiles/guppy/program/ont-guppy-cpu_3.1.5/data/template_r9.4.1_450bps_hac.jsn
input path:         data/0.fast5
save path:          result/1.basecalled
chunk size:         1000
chunks per runner:  1000
minimum qscore:     8
records per file:   0
fastq compression:  ON
num basecallers:    1
cpu mode:           ON
threads per caller: 7

Found 1 fast5 files to process.
Init time: 1531 ms

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
Caller time: 321538 ms, Samples called: 12682235, samples/s: 39442.4
Finishing up any open output files.
Basecalling completed successfully.
Basecalling is done.
------------------



real	5m32.988s
user	36m39.953s
sys	0m10.707s


---

### 2.Adapter_trimming

#### Porechop
Adapters trimming

Porechop is a tool for finding and removing adapters from Oxford Nanopore reads. Adapters on the ends of reads are trimmed off, and when a read has an adapter in its middle, it is treated as chimeric and chopped into separate reads. Porechop performs thorough alignments to effectively find adapters, even at low sequence identity.

Porechop also supports demultiplexing of Nanopore reads that were barcoded with the Native Barcoding Kit, PCR Barcoding Kit or Rapid Barcoding Kit.

<pre>
Usage: porechop -i INPUT [-o OUTPUT] [-t THREADS] [-b BARCODE_DIR] [-v VERBOSITY] 

Main options:
  -i INPUT, --input INPUT          
                       FASTA/FASTQ of input reads or a directory which will be recursively searched for FASTQ files (required)
  -o OUTPUT, --output OUTPUT       
                       Filename for FASTA or FASTQ of trimmed reads (if not set, trimmed
  -t THREADS, --threads THREADS    
                       Number of threads to use for adapter alignment (default: 4)
  -v VERBOSITY, --verbosity VERBOSITY
                       Level of progress information: 0 = none, 1 = some, 2 = lots, 3 = full - output will go to stdout if reads are saved to a file and stderr if reads are printed to stdout (default: 1)
                       
Barcode binning settings:
  Control the binning of reads based on barcodes (i.e. barcode demultiplexing)
  -b BARCODE_DIR, --barcode_dir BARCODE_DIR
                       Reads will be binned based on their barcode and saved to separate files in this directory (incompatible with --output)
</pre>

In [2]:
%%bash
in_basecalledDir=result/1.basecalled
out_filtOutDir=result/2.reads_filter
mkdir -p $out_filtOutDir
find $in_basecalledDir/ -name "*.fastq.gz" | grep pass \
  | xargs -I{} cat {} > $out_filtOutDir/reads.fastq.gz

In [3]:
%%bash
in_fq=result/2.reads_filter/reads.fastq.gz
out_fqPchop=result/2.reads_filter/reads.pchop.fastq.gz
out_pchopLog=result/2.reads_filter/porechop.log
time porechop -i $in_fq -o $out_fqPchop -t 3 -v 2 > $out_pchopLog
echo "Porechop is done."
echo "------------------"


Porechop is done.
------------------



real	0m1.432s
user	0m1.786s
sys	0m0.169s


### 3.reads_filter

#### NanoFilt 
Filter out low quality reads or particular cut-off

You can use NanoFilt to filter out read according to sequenceing quality or read length.Reads filter and/or trim pipeline

<pre>
usage: cat reads.fastq | NanoFilt [-l LENGTH] [-q QUALITY] [-s SUMMARY] > reads.filt.fastq

Perform quality and/or length and/or GC filtering of (long read) fastq data.
Options for filtering reads on.:
  -l LENGTH, --length LENGTH
                        Filter on a minimum read length
  -q QUALITY, --quality QUALITY
                        Filter on a minimum average read quality score
Input options.:
  -s SUMMARY, --summary SUMMARY
                        Use summary file for quality scores
</pre>

In [5]:
%%bash
in_fqPchop=result/2.reads_filter/reads.pchop.fastq.gz
in_seqSummary=result/1.basecalled/sequencing_summary.txt
out_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
time gunzip -c $in_fqPchop | NanoFilt -l 1000 -q 10 -s $in_seqSummary \
  | gzip > $out_fqPchopFilt


real	0m0.697s
user	0m0.900s
sys	0m0.135s


### 3.Blast
To map long read against reference genome. 

#### Python script to get top 5 longest reads from fastq

<pre>
usage: topLenFasta.py [-h] [-i <fastq>] [-n int] [--split]

get sequence length from fasta file

optional arguments:
  -h, --help            show this help message and exit
  -i <fastq>, --infile <fastq>
                        input file
  -n int, --topNumber int
                        Number of top reads
  --split               split fasta in to single file
 </pre>

In [9]:
%%bash
mkdir -p result/3.blast
in_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
out_longReads=result/3.blast
time python scripts/topLenFasta.py -n 5 -i $in_fqPchopFilt -o $out_longReads --split

get top 5 reads 
Done



real	0m0.561s
user	0m0.704s
sys	0m0.125s


---

### 4.Assembly

### Read summary

In [10]:
%%bash
cp data/2.reads_filter/reads.pchop.filt.fastq.gz \
   result/2.reads_filter/reads.pchop.filt.fastq.gz

In [13]:
%%bash
mkdir -p result/4.assembly
in_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
assembly-stats -t <(gunzip -c $in_fqPchopFilt) | cut -f1-5 | column -t

filename    total_length  number  mean_length  longest
/dev/fd/63  60882258      1488    40915.50     160588


### Assembly using Minimap/Miniasm/Racon
usage: bash run_minimap_miniasm_racon.sh [threads] [fastq] [output]

#### Step1: Overlap with Minimap

#### Step2: Layout with Miniasm

#### Step3: Consensus with Racon


In [18]:
%%bash
in_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
out_assembly=result/4.assembly/miniasm
echo "Run: run_minimap_miniasm_racon.sh"
# time bash scripts/run_minimap_miniasm_racon.sh 3 $input5 $output5
fastqFile=$in_fqPchopFilt
out=$out_assembly
thread=4
bname_fa=$(basename $fastqFile .fastq)
mkdir -p $out/miniasm
pafFile=$out/miniasm/$bname_fa.paf.gz

###################
echo "Step1: Overlap..."
###################
## fast all-against-all overlap of raw reads
echo minimap2 -x ava-ont -t $thread $fastqFile $fastqFile "|" gzip -1 ">" $pafFile
minimap2 -x ava-ont -t $thread $fastqFile $fastqFile | gzip -1 > $pafFile
echo -e "  Done\n"
#--------------------------------------------------------------------------

###################
echo "Step2: Layout..."
###################
## using miniasm, this simply concatenates pieces of read sequences 
## to generate the final
gDraft1=$out/$bname_fa.gdraft1.gfa
gDraft1_fa=$out/$bname_fa.gdraft1.fasta

echo miniasm -f $fastqFile $pafFile ">" $gDraft1
miniasm -f $fastqFile $pafFile > $gDraft1
echo -e "  Done\n"

## convert miniasm draft1 result to fasta
awk '$1 ~/S/ {print ">"$2"\n"$3}' $gDraft1 > $gDraft1_fa
#--------------------------------------------------------------------------

###################
echo "Step3: Consensus..."
###################

############
## Round 1
############
## align reads to the 1st draft of genome
## mapping the raw reads back to the assembly using minimap again
aln2gDraft1=$out/miniasm/aln.$bname_fa.vs.gdraft1.paf.gz
echo minimap2 -x map-ont -t $thread $gDraft1_fa $fastqFile "|" gzip -1 ">" $aln2gDraft1
minimap2 -x map-ont -t $thread $gDraft1_fa $fastqFile | gzip -1 > $aln2gDraft1

## using racon ('rapid consensus') for consensus calling
raconsensus1=$out/miniasm/$bname_fa.racon.draft1.consensus.fasta
echo racon -t $thread $fastqFile $aln2gDraft1 $gDraft1_fa ">" $raconsensus1
racon -t $thread $fastqFile $aln2gDraft1 $gDraft1_fa > $raconsensus1
echo -e "  Done round1...\n"

############
## Round 2
############
## perform the racon step at least twice
## re-align reads to consensus assembly data
aln2gDraft2=$out/miniasm/aln.$bname_fa.vs.gdraft2.paf.gz
echo minimap2 -x map-ont -t $thread $raconsensus1 $fastqFile "|" gzip -1 ">" $aln2gDraft2
minimap2 -x map-ont -t $thread $raconsensus1 $fastqFile | gzip -1 > $aln2gDraft2

## using racon ('rapid consensus') for consensus calling
raconsensus2=$out/$bname_fa.assembly.consensus.fasta
echo racon -t $thread $fastqFile $aln2gDraft2 $raconsensus1 ">" $raconsensus2
racon -t $thread $fastqFile $aln2gDraft2 $raconsensus1 > $raconsensus2
echo -e "  Done round2...\n"
rm -r $out/miniasm

Run: run_minimap_miniasm_racon.sh
Step1: Overlap...
minimap2 -x ava-ont -t 4 result/2.reads_filter/reads.pchop.filt.fastq.gz result/2.reads_filter/reads.pchop.filt.fastq.gz | gzip -1 > result/4.assembly/miniasm/miniasm/reads.pchop.filt.fastq.gz.paf.gz
  Done

Step2: Layout...
miniasm -f result/2.reads_filter/reads.pchop.filt.fastq.gz result/4.assembly/miniasm/miniasm/reads.pchop.filt.fastq.gz.paf.gz > result/4.assembly/miniasm/reads.pchop.filt.fastq.gz.gdraft1.gfa
  Done

Step3: Consensus...
minimap2 -x map-ont -t 4 result/4.assembly/miniasm/reads.pchop.filt.fastq.gz.gdraft1.fasta result/2.reads_filter/reads.pchop.filt.fastq.gz | gzip -1 > result/4.assembly/miniasm/miniasm/aln.reads.pchop.filt.fastq.gz.vs.gdraft1.paf.gz
racon -t 4 result/2.reads_filter/reads.pchop.filt.fastq.gz result/4.assembly/miniasm/miniasm/aln.reads.pchop.filt.fastq.gz.vs.gdraft1.paf.gz result/4.assembly/miniasm/reads.pchop.filt.fastq.gz.gdraft1.fasta > result/4.assembly/miniasm/miniasm/reads.pchop.filt.fastq.gz.r

[M::mm_idx_gen::2.075*1.14] collected minimizers
[M::mm_idx_gen::2.390*1.51] sorted minimizers
[M::main::2.390*1.51] loaded/built the index for 1488 target sequence(s)
[M::mm_mapopt_update::2.535*1.48] mid_occ = 62
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 1488
[M::mm_idx_stat::2.628*1.46] distinct minimizers: 8011485 (75.63% are singletons); average occurrences: 2.591; average spacing: 2.933
[M::worker_pipeline::10.357*3.11] mapped 1488 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -x ava-ont -t 4 result/2.reads_filter/reads.pchop.filt.fastq.gz result/2.reads_filter/reads.pchop.filt.fastq.gz
[M::main] Real time: 10.453 sec; CPU: 32.275 sec; Peak RSS: 0.770 GB
[M::main] ===> Step 1: reading read mappings <===
[M::ma_hit_read::0.075*1.02] read 60270 hits; stored 80266 hits and 1488 sequences (60882258 bp)
[M::main] ===> Step 2: 1-pass (crude) read selection <===
[M::ma_hit_sub::0.082*1.02] 1488 query sequences remain after sub
[M::ma_hit_cut::0.083*1.02]

#### get example of non-polished sequence 

In [ ]:
%%bash
in_assemDraft1=result/4.assembly/miniasm/reads.pchop.filt.fastq.gz.gdraft1.fasta
out_nonPolishAssem=result/4.assembly/miniasm/non-polished.chr.fasta
samtools faidx $in_assemDraft1 utg000001c:1-20000 \
 | bioawk -c fastx '{print ">"$name":non-polished\n"$seq}' > $out_nonPolishAssem

#### get example of polished sequence


In [ ]:
%%bash
in_assem=result/4.assembly/miniasm/reads.pchop.filt.fastq.gz.assembly.consensus.fasta
out_polishAssem=result/4.assembly/miniasm/polished.chr.fasta
samtools faidx $in_assem utg000001c:1-20000 \
 | bioawk -c fastx '{print ">"$name":polished\n"$seq}' > $out_polishAssem

### Assembly using Unicycler
https://github.com/rrwick/Unicycler

usage: unicycler -t [threads] -l [fastq] -o [output]

In [16]:
%%bash
in_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
out_unicycler=result/4.assembly/unicycler
time unicycler -t 4 -l $in_fqPchopFilt -o $out_unicycler


Starting Unicycler (2019-08-26 00:40:12)
    Welcome to Unicycler, an assembly pipeline for bacterial genomes. Since you
provided only long reads, Unicycler will assemble the reads with miniasm and
then run repeated polishing rounds using Racon.
    For more information, please see https://github.com/rrwick/Unicycler

Command: /Users/piroonjenjaroenpun/anaconda3/envs/ont_assembly/bin/unicycler -t 4 -l result/2.reads_filter/reads.pchop.filt.fastq.gz -o result/4.assembly/unicycler

Unicycler version: v0.4.8
Using 4 threads

Making output directory:
  /Users/piroonjenjaroenpun/Desktop/project/workshop/19_ONT_at_KMUTT/ont_workshop/result/4.assembly/unicycler

Dependencies:
  Program         Version   Status  
  spades.py                 not used
  racon           1.4.3     good    
  makeblastdb     2.6.0+    good    
  tblastn         2.6.0+    good    
  bowtie2-build             not used
  bowtie2                   not used
  samtools                  not used
  java                   


real	3m24.774s
user	10m14.928s
sys	0m13.125s


### To view Graphical Fragment Assembly (GFA) Format

In [ ]:
%%bash
source /opt/anaconda3/bin/activate bandage
in_assemGFA=result/4.assembly/unicycler/assembly.gfa
out_assemDraw=result/4.assembly/unicycler/assembly_unicycler_draw.png
Bandage image $in_assemGFA $out_assemDraw

In [1]:
from IPython.core.display import Image, display
display(Image('result/4.assembly/unicycler/assembly_unicycler_draw.png', width=750))


FileNotFoundError: No such file or directory: 'result/4.assembly/unicycler/assembly_draw.png'

FileNotFoundError: No such file or directory: 'result/4.assembly/unicycler/assembly_draw.png'

<IPython.core.display.Image object>

---

### Flye

In [20]:
%%bash
in_fqPchopFilt=result/2.reads_filter/reads.pchop.filt.fastq.gz
out_flye=result/4.assembly/flye
time flye --nano-raw $in_fqPchopFilt --out-dir $out_flye --genome-size 2.9m --threads 4

[2019-08-26 00:57:14] INFO: Starting Flye 2.5-release
[2019-08-26 00:57:14] INFO: >>>STAGE: configure
[2019-08-26 00:57:14] INFO: Configuring run
[2019-08-26 00:57:15] INFO: Total read length: 60882258
[2019-08-26 00:57:15] INFO: Input genome size: 2900000
[2019-08-26 00:57:15] INFO: Estimated coverage: 20
[2019-08-26 00:57:15] INFO: Reads N50/N90: 40828 / 28463
[2019-08-26 00:57:15] INFO: Minimum overlap set to 5000
[2019-08-26 00:57:15] INFO: Selected k-mer size: 15
[2019-08-26 00:57:15] INFO: >>>STAGE: assembly
[2019-08-26 00:57:15] INFO: Assembling disjointigs
[2019-08-26 00:57:15] INFO: Reading sequences
[2019-08-26 00:57:15] INFO: Generating solid k-mer index
[2019-08-26 00:57:20] INFO: Counting k-mers (1/2):
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2019-08-26 00:57:23] INFO: Counting k-mers (2/2):
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2019-08-26 00:57:28] INFO: Filling index table
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2019-08-26 00:58:11] INFO: Extending rea

In [ ]:
%%bash
source /opt/anaconda3/bin/activate bandage
in_assemGFA=result/4.assembly/flye/assembly_graph.gfa
out_assemDraw=result/4.assembly/flye/assembly_flye_draw.png
Bandage image $in_assemGFA $out_assemDraw

In [1]:
from IPython.core.display import Image, display
display(Image('result/4.assembly/flye/assembly_flye_draw.png', width=750))


FileNotFoundError: No such file or directory: 'result/4.assembly/unicycler/assembly_draw.png'

FileNotFoundError: No such file or directory: 'result/4.assembly/unicycler/assembly_draw.png'

<IPython.core.display.Image object>

In [27]:
%%bash
in_unicyclerAssem=result/4.assembly/unicycler/assembly.fasta
in_flyeAssem=result/4.assembly/flye/assembly.fasta
mkdir -p result/5.comparative
out_uni_vs_flye_paf=result/5.comparative/uni_vs_flye.paf
minimap2 -x asm5 $in_unicyclerAssem $in_flyeAssem > $out_uni_vs_flye_paf


[M::mm_idx_gen::0.069*1.02] collected minimizers
[M::mm_idx_gen::0.078*1.24] sorted minimizers
[M::main::0.078*1.24] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.082*1.23] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.085*1.22] distinct minimizers: 281517 (99.09% are singletons); average occurrences: 1.019; average spacing: 10.006
[M::worker_pipeline::0.232*1.07] mapped 1 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -x asm5 result/4.assembly/unicycler/assembly.fasta result/4.assembly/flye/assembly.fasta
[M::main] Real time: 0.247 sec; CPU: 0.262 sec; Peak RSS: 0.044 GB


In [29]:
%%bash
Rscript scripts/pafCoordsDotPlotly.R -i result/5.comparative/uni_vs_flye.paf \
  -o result/5.comparative/uni_vs_flye -t -m 500 -q 500000 -k 7 -l -p 7

PARAMETERS:
input (-i): result/5.comparative/uni_vs_flye.paf
output (-o): result/5.comparative/uni_vs_flye
minimum query aggregate alignment length (-q): 5e+05
minimum alignment length (-m): 500
plot size (-p): 7
show horizontal lines (-l): TRUE
number of reference chromosomes to keep (-k): 7
show % identity (-s): FALSE
show % identity for on-target alignments only (-t): FALSE
produce interactive plot (-x): TRUE
reference IDs to keep (-r): 

Number of alignments: 2
Number of query sequences: 1

After filtering... Number of alignments: 2
After filtering... Number of query sequences: 1

No traceback available 


Error in normalizePath(basepath, "/", TRUE) : 
  path[1]="result/5.comparative": No such file or directory
Calls: <Anonymous> ... pandoc_save_markdown -> lapply -> FUN -> <Anonymous> -> normalizePath
